In [ ]:
!pip install git+https://github.com/mhamilton723/FeatUp

import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [3]:
def closest_multiple(n, mult):
    """
    Finds the closest multiple of `mult` less than or equal to `n`.
    """
    return n - (n % mult)

def transform(img_path, patch_size=14):
    """
    Preprocess an image to tensor form.
    - Resizes to a fixed size (e.g., 448x448).
    - Normalizes using ImageNet mean and std.
    """
    bgr = cv2.imread(img_path)
    height, width, _ = bgr.shape
    new_h = closest_multiple(height, patch_size)
    new_w = closest_multiple(width, patch_size)
    resized = cv2.resize(bgr, (448, 448))

    rgb = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB).astype(np.float32)
    rgb /= 255.0

    mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
    rgb = (rgb - mean) / std

    rgb_chw = rgb.transpose(2, 0, 1)  # (3, H, W)
    tensor = torch.from_numpy(rgb_chw).unsqueeze(0)  # (1, 3, H, W)
    return tensor


In [5]:
def get_featup_pca_images(img_path, upsampler, device="cpu", patch_size=14):
    """
    Applies PCA on feature maps to reduce dimensionality for visualization.
    """
    # 1) Preprocess image
    img_tensor = transform(img_path, patch_size=patch_size).to(device)

    # 2) Generate feature maps with FeatUp model
    with torch.no_grad():
        feats_upsampled = upsampler(img_tensor)
    B, C, H_up, W_up = feats_upsampled.shape

    # 3) Perform PCA
    feats_2d = feats_upsampled.permute(0, 2, 3, 1).reshape(-1, C).cpu().numpy()
    pca = PCA(n_components=3)
    feats_pca_3d = pca.fit_transform(feats_2d)
    scaler = MinMaxScaler()
    feats_pca_3d = scaler.fit_transform(feats_pca_3d)

    # 4) Reshape PCA result back to image
    feats_pca_3d = feats_pca_3d.reshape(H_up, W_up, 3)
    feats_pca_255 = (feats_pca_3d * 255).astype(np.uint8)

    # 5) Prepare original image for comparison
    input_img = img_tensor[0].permute(1, 2, 0).cpu().numpy()
    mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
    inp = np.clip(input_img * std + mean, 0.0, 1.0)
    inp_255 = (inp * 255).astype(np.uint8)

    return inp_255, feats_pca_255

In [6]:
def visualize_images_grid_featup(img_paths, upsampler, device="cpu", patch_size=14):
    """
    Visualizes input images and their PCA-transformed feature maps side by side.
    """
    results = []
    for path in img_paths:
        inp_255, feats_pca_255 = get_featup_pca_images(
            path, upsampler, device=device, patch_size=patch_size
        )
        results.append((path, inp_255, feats_pca_255))

    # Create a grid to display images
    n_images = len(results)
    fig, axes = plt.subplots(nrows=n_images, ncols=2, figsize=(10, 5 * n_images))

    if n_images == 1:
        axes = [axes]  # Handle case with a single image

    for row_idx, (path, inp_img, feat_img) in enumerate(results):
        ax_left, ax_right = axes[row_idx]
        ax_left.imshow(inp_img)
        ax_left.axis("off")

        ax_right.imshow(feat_img)
        ax_right.set_title("FeatUp + PCA")
        ax_right.axis("off")

    plt.tight_layout()
    plt.show()


In [ ]:
# Example image paths
image_paths = [
    "/content/splits_final_deblurred/train/data/04_frame_036100.PNG",
    "/content/splits_final_deblurred/train/data/04_frame_036200.PNG",
]

upsampler = torch.hub.load("mhamilton723/FeatUp", "dinov2", use_norm=False)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Visualize images and PCA-transformed features
visualize_images_grid_featup(image_paths, upsampler, device=device, patch_size=14)